In [1]:
import numpy as np
import scipy.io
import json

import sys
sys.path.append('/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/code')
from dn_utils.behavioral_models import load_behavioral_data, estimate_values_pd

In [2]:
beh_path = "/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/" \
           "data/main_fmri_study/sourcedata/behavioral" 
beh, meta = load_behavioral_data(root=beh_path)
n_subjects = beh.shape[0]
n_conditions = beh.shape[1]
n_trials = beh.shape[2]

Shape of beh array: (32, 2, 110, 21)
Conditions [(0, 'rew'), (1, 'pun')]
Columns: [(0, 'block'), (1, 'rwd'), (2, 'magn_left'), (3, 'magn_right'), (4, 'response'), (5, 'rt'), (6, 'won_bool'), (7, 'won_magn'), (8, 'acc_after_trial'), (9, 'onset_iti'), (10, 'onset_iti_plan'), (11, 'onset_iti_glob'), (12, 'onset_dec'), (13, 'onset_dec_plan'), (14, 'onset_dec_glob'), (15, 'onset_isi'), (16, 'onset_isi_plan'), (17, 'onset_isi_glob'), (18, 'onset_out'), (19, 'onset_out_plan'), (20, 'onset_out_glob')]


In [3]:
def generate_responses_pd(beh, meta, subject, condition, 
                         alpha_plus, alpha_minus, beta):
    '''Creates responses from artificial subject using specified parameter 
    values. Note that it uses already existing task data (like trial-wise reward
    magnitudes and correct boxes) but it is independent of the collected 
    behavioral data.
    
    Args:
        beh (np.array): aggrgated behavioral response
        meta (dict): description of beh array coding
        subject (int): subject index
        condition (int): task condition index 
        alpha_plus (float): learning rate for positive PE
        alpha_minus (float): learning rate for negative PE
        beta (float): inverse-temperature parameter
        
    Returns:
        Vector of artificial responses. Values coding left and right simulated 
        button presses are -1 and 1. 
    '''


    n_trials = beh.shape[2]
    val, util, magn = (np.zeros((n_trials, 2)) for _ in range(3))
    p, response = (np.zeros((n_trials)) for _ in range(2))
    val[0] = [.5, .5] # Initial belefis (agnostic)

    rwd = np.copy(beh[subject, condition, :, meta['dim4'].index('rwd')]) #BCI

    magn[:, 0] = beh[subject, condition, :, meta['dim4'].index('magn_left')]
    magn[:, 1] = beh[subject, condition, :, meta['dim4'].index('magn_right')]


    for t in range(0, n_trials):

        # Calculate expected utility 
        util[t, :] = np.multiply(magn[t, :], val[t, :])

        # Pass it into softmax & generate response
        p[t] = np.exp(beta*util[t, 1]) / np.sum(np.exp(beta*util[t, :]))
        response[t] = np.random.choice([-1, 1], p=[1-p[t], p[t]])

        # Use correct learning rate for trial
        if condition == 0:
            alpha = alpha_plus * (rwd[t] == response[t]) \
                  + alpha_minus * (rwd[t] != response[t])
        else:
            alpha = alpha_plus * ((-1)*rwd[t] == response[t]) \
                  + alpha_minus * ((-1)*rwd[t] != response[t])

        # Update val before next trial (ignore last update)
        if t < n_trials-1: 
            val[t+1, 1] = val[t, 1] + alpha * ((rwd[t] + 1)/2 - val[t, 1])
            val[t+1, 0] = val[t, 0] + alpha * ((-rwd[t] + 1)/2 - val[t, 0])


    return response

### Create synthetic responses for different RL parameters
Here, artificial responses are created for different values of possible RL parameters. Task data is pooled from beh matrix (it is not generated from scratch, but reuse existing data). Note that subject behavioral responses **are not used** at all.

In [266]:
n_gridpoints_alpha_plus = 10
n_gridpoints_alpha_minus = 10
n_gridpoints_beta = 3

grid_alpha_plus = np.linspace(.01, .99, n_gridpoints_alpha_plus)
grid_alpha_minus = np.linspace(.01, .99, n_gridpoints_alpha_minus)
grid_beta = np.linspace(.05, .15, n_gridpoints_beta)

response_synthetic = np.zeros(
    (n_gridpoints_alpha_plus, n_gridpoints_alpha_minus, n_gridpoints_beta,
     n_conditions, n_trials)
)

for ix_ap, ap in enumerate(grid_alpha_plus):
    for ix_am, am in enumerate(grid_alpha_minus):
        for ix_bt, bt in enumerate(grid_beta):
            
            for con in [0, 1]:
                response_synthetic[ix_ap, ix_am, ix_bt, con, :] = \
                    generate_responses_pd(beh, meta, 
                                          0, con, 
                                          ap, am, bt)

In [271]:
# Save synthetic responses to *.mat format
scipy.io.savemat(
    'matjags-dn/data/parameter_recovery/response_synthetic_sub_0.mat', 
    {'response_synthetic': response_synthetic}, appendmat=True)

### Refactoring:
1. create additional folder structure within matjags-dn/data
2. consistent variable names in behavioral_models.py
3. test to behavioral_models.py (?)
4. create MY rules to follow (naming conventions, BCI vs correctI distinction)